In [4]:
from openai import OpenAI
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [13]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('../larrys_database.db')

# Get a cursor object
cur = conn.cursor()

# Execute a query to get all table names in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# For each table
for table_name in tables:
    print(table_name)
    if '.db' in table_name[0]:
        continue
    table_name = table_name[0]
    table = pd.read_sql_query("SELECT * from %s" % table_name, conn)
    
    # Write the data of each table to a CSV file
    table.to_csv('../data/'+table_name + '.csv', index=False)

# Close the connection
conn.close()

('voice_log',)
('points',)
('larrys_database.db',)


In [78]:
voice_log_file = client.files.create(
    file=open("../data/voice_log.csv", "rb"),
    purpose="assistants"
)
points_file = client.files.create(
    file=open("../data/points.csv", "rb"),
    purpose="assistants"
)

my_assistant = client.beta.assistants.create(
    instructions="You are an assistant to my discord bot called Larry's Bot that lives in Larry's Gym. You will provide interesting tidbits of information contained in points.csv and voice_log.csv (both of which are tables in the database called larrys_database.db), by reading the data in pandas and evaluating interesting trends in points. In general, try to look at interesting trends by individual users, excluding 'Larry's Bot' from your analyses. Make sure to sort the dates by the most recent days and give analyses based on the most recent days with occasional callbacks to further out days. Make sure to provide the information with some flair and a sense of competitiveness.",
    name="Larry's Assistant",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-0125-preview",
    file_ids=[voice_log_file.id, points_file.id]
)

print(my_assistant)

Assistant(id='asst_K7rd8jZdokf4g1pyWaGDz5hl', created_at=1712120410, description=None, file_ids=['file-URlqevL5JOu7nrNEJotVEtNf', 'file-c1agw2CS6Az1RBiwkzBynIEt'], instructions="You are an assistant to my discord bot called Larry's Bot that lives in Larry's Gym. You will provide interesting tidbits of information contained in points.csv and voice_log.csv (both of which are tables in the database called larrys_database.db), by reading the data in pandas and evaluating interesting trends in points. In general, try to look at interesting trends by individual users, excluding 'Larry's Bot' from your analyses. Make sure to sort the dates by the most recent days and give analyses based on the most recent days with occasional callbacks to further out days. Make sure to provide the information with some flair and a sense of competitiveness.", metadata={}, model='gpt-4-0125-preview', name="Larry's Assistant", object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')])


In [79]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_0Ykpup9hyVOqhY0CoLBoWY7l', created_at=1712120411, metadata={}, object='thread')


In [80]:
thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="assistant",
    content="Let's recap the most recent day of data, shall we?"
)

In [81]:
thread_message

Message(id='msg_wt2Zpg9vOxXonDrnyoVMrnca', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Let's recap the most recent day of data, shall we?"), type='text')], created_at=1712120411, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id=None, status=None, thread_id='thread_0Ykpup9hyVOqhY0CoLBoWY7l')

In [82]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=my_assistant.id,
)

In [83]:
# run

In [84]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )

In [85]:
messages

SyncCursorPage[Message](data=[Message(id='msg_7ykmAUy1rBcMplC8yCNZC7am', assistant_id='asst_K7rd8jZdokf4g1pyWaGDz5hl', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="In the heated battle of consistency over the past week in Larry's Gym, the standings reveal a fierce competition:\n\n- **dinkstar**, the embodiment of determination, championed with **7** gym entrances!\n- Hot on their heels, **bemno** showed formidable commitment, entering **5** times.\n- Not to be overlooked, **kyboydigital** and the ever-reliable gym bot, both stepped in **4** times, sharing the bronze spotlight.\n\nTruly, this past week has been a testament to the unyielding spirit of Larry's Gym members. Let's keep the fire burning and seek out more intriguing insights, shall we?"), type='text')], created_at=1712120496, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_MUEnPoy9QazkzkCHOZPnN3cd', status=None, thre

In [77]:
pd.read_csv('../data/points.csv').sort_values(by='day',ascending=False)

,name,id,points_awarded,day,type
523,dinkstar,353332394541187074,50.000000,2024-03-30,DURATION
522,Larry's Gym Bot,1191145890925260810,0.384693,2024-03-30,DURATION
521,dinkstar,353332394541187074,48.183295,2024-03-30,ON TIME
520,Larry's Gym Bot,1191145890925260810,32.218400,2024-03-30,ON TIME
519,kyboydigital,639283802308476970,1.750946,2024-03-29,DURATION
...,...,...,...,...,...
1,bemno,369989877229682688,29.037037,2024-01-02,ON TIME
5,bemno,369989877229682688,48.611111,2024-01-02,DURATION
6,jam4bears,390403088722165762,50.000000,2024-01-02,DURATION
7,kyboydigital,639283802308476970,49.240741,2024-01-02,DURATION
